Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, записывающую собранные вакансии в созданную БД

In [55]:
from bs4 import BeautifulSoup as bs  
from bs4 import BeautifulStoneSoup    
import bs4     
import requests
import json
import pandas as pd
import numpy as np
import re
from pymongo import MongoClient
import ast


In [2]:
def getPage(page = 0):
    """
    Создаем метод для получения страницы со списком вакансий.
    Аргументы:
        page - Индекс страницы, начинается с 0. Значение по умолчанию 0, т.е. первая страница
    """
     
    # Справочник для параметров GET-запроса
    params = {
        'text': 'NAME:Data Scientist', # Текст фильтра. В имени должно быть слово "Аналитик"
        'area': 1, # Поиск ощуществляется по вакансиям города Москва
        'page': page, # Индекс страницы поиска на HH
        'per_page': 100 # Кол-во вакансий на 1 странице
    }
     
     
    req = requests.get('https://api.hh.ru/vacancies', params) # Посылаем запрос к API
    data = req.content.decode() # Декодируем его ответ, чтобы Кириллица отображалась корректно
    req.close()
    return data

In [68]:
def load_data(p):
    jsObj = json.loads(getPage(0))
    my_df = pd.DataFrame.from_dict(jsObj['items'], orient='columns')
    # Считываем первые 2000 вакансий
    for page in range(1, 20):

        # Преобразуем текст ответа запроса в справочник Python
        jsObj = json.loads(getPage(page))
        my_df_ = pd.DataFrame.from_dict(jsObj['items'], orient='columns')
        my_df = pd.concat([my_df, my_df_], axis=0)
        # Проверка на последнюю страницу, если вакансий меньше 2000
        if (jsObj['pages'] - page) <= 1:
            break

    print('Старницы поиска собраны')
    return my_df

In [7]:
client = MongoClient('localhost', 27017)

In [8]:
db = client['HH_database']

In [77]:
collection = db.HH_API_collection

In [69]:
my_df = load_data(1)

Старницы поиска собраны


In [70]:
for i, row in my_df.iterrows():
    if row['salary'] != None:
        for e in row['salary']:
            my_df.loc[i, e] = row['salary'][e]
my_df.rename(columns={'from': 'salary_from', 'to':'salary_to', 'currency':'salary_currency', 'gross':'salary_gross'}, inplace=True)

In [71]:
my_df.head(2)

,id,premium,name,department,has_test,response_letter_required,area,salary,type,address,...,contacts,schedule,working_days,working_time_intervals,working_time_modes,accept_temporary,salary_from,salary_to,salary_currency,salary_gross
0,48434078,False,Data Scientist,None,False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...","{'from': 180000, 'to': 250000, 'currency': 'RU...","{'id': 'open', 'name': 'Открытая'}",None,...,None,"{'id': 'remote', 'name': 'Удаленная работа'}",[],[],[],False,180000.0,250000.0,RUR,False
1,48703955,False,Junior Data Scientist,None,False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...",None,"{'id': 'open', 'name': 'Открытая'}",None,...,None,"{'id': 'fullDay', 'name': 'Полный день'}",[],[],[],False,NaN,3000.0,USD,True


In [72]:
db.collection.insert_many(my_df.to_dict('records'))

In [88]:
a = db.collection.find({})

In [89]:
a

In [90]:
for doc in a:
    print(doc)
    print('**************************')
    break

{'_id': ObjectId('616468d5d743c782d93ff30b'), 'id': '48434078', 'premium': False, 'name': 'Data Scientist', 'department': None, 'has_test': False, 'response_letter_required': False, 'area': {'id': '1', 'name': 'Москва', 'url': 'https://api.hh.ru/areas/1'}, 'salary': {'from': 180000, 'to': 250000, 'currency': 'RUR', 'gross': False}, 'type': {'id': 'open', 'name': 'Открытая'}, 'address': None, 'response_url': None, 'sort_point_distance': None, 'published_at': '2021-10-10T12:06:52+0300', 'created_at': '2021-10-10T12:06:52+0300', 'archived': False, 'apply_alternate_url': 'https://hh.ru/applicant/vacancy_response?vacancyId=48434078', 'insider_interview': None, 'url': 'https://api.hh.ru/vacancies/48434078?host=hh.ru', 'alternate_url': 'https://hh.ru/vacancy/48434078', 'relations': [], 'employer': {'id': '1664868', 'name': 'CORE', 'url': 'https://api.hh.ru/employers/1664868', 'alternate_url': 'https://hh.ru/employer/1664868', 'logo_urls': {'original': 'https://hhcdn.ru/employer-logo-original/